<a href="https://colab.research.google.com/github/hufahamdika/BigData_MidtermExam/blob/main/Recommendation%20System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Big Data Midterm Exam
#### Recommendation Systems – Frequent Pattern Mining

<b>Nama : Abidjanna Zulfa Hamdika</b><br>
<b>NRP : 5025201197</b><br>
<b>Kelas : Big Data A</b><br>

In [1]:
# Mount Data
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# Read CSV
import pandas as pd
data = pd.read_csv('/content/gdrive/My Drive/Kuliah/Big Data/ETS/market-basket.csv', sep=';')
data.head()

<ipython-input-2-b6aa16c58970>:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/content/gdrive/My Drive/Kuliah/Big Data/ETS/market-basket.csv', sep=';')


,BillNo,Itemname,Quantity,Date,Price,CustomerID,Country
0,536365,WHITE HANGING HEART T-LIGHT HOLDER,6,01.12.2010 08:26,"2,55",17850.0,United Kingdom
1,536365,WHITE METAL LANTERN,6,01.12.2010 08:26,"3,39",17850.0,United Kingdom
2,536365,CREAM CUPID HEARTS COAT HANGER,8,01.12.2010 08:26,"2,75",17850.0,United Kingdom
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE,6,01.12.2010 08:26,"3,39",17850.0,United Kingdom
4,536365,RED WOOLLY HOTTIE WHITE HEART.,6,01.12.2010 08:26,"3,39",17850.0,United Kingdom


In [3]:
# install java and apache spark
%%capture
!sudo apt-get update --fix-missing

!apt-get install openjdk-8-jdk-headless -qq > /dev/null

!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

!mv spark-3.0.0-bin-hadoop3.2.tgz sparkkk
!tar xf sparkkk
!pip install -q findspark

In [4]:
!pip install spark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for spark: filename=spark-0.2.1-py3-none-any.whl size=58760 sha256=b720b21c12d7c847ef682031cde8fc6fb5b1ed8c7cbfa577d63a395f4e9c861d
  Stored in directory: /root/.cache/pip/wheels/bc/0f/6c/b41528ca0fd4d46513185e90da91bc1c484bba6138641c5a62
Successfully built spark


In [5]:
# create environment variables

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName('fpgrowth') \
    .getOrCreate()

spark   

In [ ]:
#files.upload()

In [6]:
data

,BillNo,Itemname,Quantity,Date,Price,CustomerID,Country
0,536365,WHITE HANGING HEART T-LIGHT HOLDER,6,01.12.2010 08:26,"2,55",17850.0,United Kingdom
1,536365,WHITE METAL LANTERN,6,01.12.2010 08:26,"3,39",17850.0,United Kingdom
2,536365,CREAM CUPID HEARTS COAT HANGER,8,01.12.2010 08:26,"2,75",17850.0,United Kingdom
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE,6,01.12.2010 08:26,"3,39",17850.0,United Kingdom
4,536365,RED WOOLLY HOTTIE WHITE HEART.,6,01.12.2010 08:26,"3,39",17850.0,United Kingdom
...,...,...,...,...,...,...,...
522059,581587,PACK OF 20 SPACEBOY NAPKINS,12,09.12.2011 12:50,"0,85",12680.0,France
522060,581587,CHILDREN'S APRON DOLLY GIRL,6,09.12.2011 12:50,"2,1",12680.0,France
522061,581587,CHILDRENS CUTLERY DOLLY GIRL,4,09.12.2011 12:50,"4,15",12680.0,France
522062,581587,CHILDRENS CUTLERY CIRCUS PARADE,4,09.12.2011 12:50,"4,15",12680.0,France


In [7]:
from google.colab import files
from pyspark.sql import functions as F
from pyspark.ml.fpm import FPGrowth

In [8]:
# change all datatype to string
data = data.astype(str)

In [9]:
# check data again
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 522064 entries, 0 to 522063
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   BillNo      522064 non-null  object
 1   Itemname    522064 non-null  object
 2   Quantity    522064 non-null  object
 3   Date        522064 non-null  object
 4   Price       522064 non-null  object
 5   CustomerID  522064 non-null  object
 6   Country     522064 non-null  object
dtypes: object(7)
memory usage: 27.9+ MB


In [10]:
data.describe()

,BillNo,Itemname,Quantity,Date,Price,CustomerID,Country
count,522064,522064,522064,522064,522064,522064,522064
unique,21663,4186,690,19641,1285,4298,30
top,573585,WHITE HANGING HEART T-LIGHT HOLDER,1,31.10.2011 14:41,"1,25",nan,United Kingdom
freq,1114,2269,147848,1114,48778,134041,487622


In [11]:
# transform it into spark dataframe
sparkdata = spark.createDataFrame(data)
basketdata = sparkdata.dropDuplicates(['BillNo', 'Itemname']).sort('BillNo')
basketdata = basketdata.groupBy("BillNo").agg(F.collect_list("Itemname")).sort('BillNo')

/content/spark-3.0.0-bin-hadoop3.2/python/pyspark/sql/pandas/conversion.py:327: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


### Min Support = 0.006, Min Confidence = 0.006

In [ ]:
#Frequent Pattern Growth – FP Growth is a method of mining frequent itemsets
fpGrowth = FPGrowth(itemsCol="collect_list(Itemname)", minSupport=0.006, minConfidence=0.006) 
model = fpGrowth.fit(basketdata)

# Display frequent itemsets.
model.freqItemsets.show()
items = model.freqItemsets
# Display generated association rules.
model.associationRules.show()
rules = model.associationRules
# transform examines the input items against all the association rules and summarize the
# consequents as prediction
model.transform(basketdata).show()
transformed = model.transform(basketdata)

+--------------------+----+
|               items|freq|
+--------------------+----+
|[FELTCRAFT BUTTER...| 471|
|[FELTCRAFT BUTTER...| 167|
|[VINTAGE CHRISTMA...| 334|
|[ROCKING HORSE RE...| 245|
|[6 RIBBONS SHIMME...| 198|
|[VICTORIAN SEWING...| 160|
|[PARTY TIME PENCI...| 133|
|[WHITE HANGING HE...|2202|
|[STRAWBERRY FAIRY...| 198|
|[CAMOUFLAGE LED T...| 160|
|[PINK LOVE HEART ...| 133|
|[JUMBO BAG RED RE...|2064|
|[JUMBO BAG RED RE...| 442|
|[JUMBO BAG VINTAG...| 470|
|[JUMBO BAG VINTAG...| 136|
|[JUMBO BAG VINTAG...| 319|
|[JUMBO BAG VINTAG...| 130|
|[JUMBO BAG VINTAG...| 154|
|[JUMBO BAG VINTAG...| 180|
|[JUMBO BAG VINTAG...| 132|
+--------------------+----+
only showing top 20 rows

+--------------------+--------------------+-------------------+------------------+
|          antecedent|          consequent|         confidence|              lift|
+--------------------+--------------------+-------------------+------------------+
|[HOT WATER BOTTLE...|[LOVE HOT WATER B...| 0.3009079

In [ ]:
# Convert the Spark DataFrame back to a Pandas DataFrame using Arrow
result_pdf = items.select("*").toPandas()
result_pdf.head()

,items,freq
0,[FELTCRAFT BUTTERFLY HEARTS],471
1,"[FELTCRAFT BUTTERFLY HEARTS, FELTCRAFT 6 FLOWE...",167
2,[VINTAGE CHRISTMAS STOCKING],334
3,[ROCKING HORSE RED CHRISTMAS],245
4,[6 RIBBONS SHIMMERING PINKS],198


In [ ]:
result_pdf.to_excel('result_pdfItemsFreq_supp=0.006_conf=0.006.xlsx')

In [ ]:
rules_pdf = rules.select("*").toPandas()
rules_pdf.head()

,antecedent,consequent,confidence,lift
0,[HOT WATER BOTTLE KEEP CALM],[LOVE HOT WATER BOTTLE],0.300908,14.018426
1,[HOT WATER BOTTLE KEEP CALM],[CHOCOLATE HOT WATER BOTTLE],0.345006,8.940042
2,[HOT WATER BOTTLE KEEP CALM],[PAPER CHAIN KIT 50'S CHRISTMAS],0.216602,4.105201
3,[HOT WATER BOTTLE KEEP CALM],[GARDENERS KNEELING PAD KEEP CALM],0.225681,5.402128
4,[HOT WATER BOTTLE KEEP CALM],[SCOTTIE DOG HOT WATER BOTTLE],0.255512,7.292705


In [ ]:
rules_pdf.to_excel('rules_pdfAnteConseConfLift_2upp=0.006_conf=0.006.xlsx')

In [ ]:
transformed_pdf = transformed.select("*").toPandas()
transformed_pdf.head()

,BillNo,collect_list(Itemname),prediction
0,536365,"[KNITTED UNION FLAG HOT WATER BOTTLE, SET 7 BA...","[JUMBO BAG RED RETROSPOT, REGENCY CAKESTAND 3 ..."
1,536366,"[HAND WARMER UNION JACK, HAND WARMER RED POLKA...","[HAND WARMER RED LOVE HEART, HAND WARMER RED R..."
2,536367,"[BOX OF VINTAGE JIGSAW BLOCKS, FELTCRAFT PRINC...","[POPPY'S PLAYHOUSE LIVINGROOM, POPPY'S PLAYHOU..."
3,536368,"[YELLOW COAT RACK PARIS FASHION, BLUE COAT RAC...","[JAM JAR WITH PINK LID, SMALL HEART MEASURING ..."
4,536369,[BATH BUILDING BLOCK WORD],[HOME BUILDING BLOCK WORD]


In [ ]:
transformed_pdf.to_excel('transformed_pdfSalesTransactionIDCollectListPred.xlsx')

In [ ]:
#Frequent Pattern Growth – FP Growth is a method of mining frequent itemsets
fpGrowth = FPGrowth(itemsCol="collect_list(Itemname)", minSupport=0.003, minConfidence=0.003) 
model = fpGrowth.fit(basketdata)

# Display frequent itemsets.
model.freqItemsets.show()
items = model.freqItemsets
# Display generated association rules.
model.associationRules.show()
rules = model.associationRules
# transform examines the input items against all the association rules and summarize the
# consequents as prediction
model.transform(basketdata).show()
transformed = model.transform(basketdata)

In [ ]:
# Convert the Spark DataFrame back to a Pandas DataFrame using Arrow
result_pdf = items.select("*").toPandas()
result_pdf.head()

In [ ]:
result_pdf.to_excel('result_pdfItemsFreq_supp=0.003_conf=0.003.xlsx')

In [ ]:
rules_pdf = rules.select("*").toPandas()
rules_pdf.head()

In [ ]:
rules_pdf.to_excel('rules_pdfAnteConseConfLift_2upp=0.006_conf=0.006.xlsx')

In [ ]:
transformed_pdf = transformed.select("*").toPandas()
transformed_pdf.head()

In [ ]:
transformed_pdf.to_excel('transformed_pdfSalesTransactionIDCollectListPred.xlsx')